<blockquote>
    <h1>Exercise 3.9</h1>
    <p>This question should be answered using the <code>Carseats</code> data set.</p>
    <ol>
        <li>Fit a multiple regression model to predict $\mathrm{Sales}$ using $\mathrm{Price}$, $\mathrm{Urban}$, and $\mathrm{US}$.</li>
        <li>Provide an interpretation of each coefficient in the model. Be careful - some of the variables in the model are qualitative!</li>
        <li>Write out the model in equation form, being careful to handle the qualitative variables properly.</li>
        <li>For which of the predictors can you reject the null hypothesis $H_0:\beta_j = 0$?</li>
        <li>On the basis of your response to the previous question, fit a smaller model that only uses the predictors for which there is evidence of association with the outcome.</li>
        <li>How well do the models in 1 and 5 fit the data?</li>
        <li>Using the model from 5, obtain $95 \%%$ confidence intervals for the coefficient(s).</li>
        <li>Is there evidence of outliers or high leverage observations in the model from 5?</li>
    </ol>
</blockquote>